In [ ]:
# --- 🚀 TORUS universal imports / helpers ---
import os, json, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import chi2, pearsonr
from scipy.special import erfcinv
np.random.seed(42)

# Safe σ helper – never let σ==0 blow up χ²
def safe_sigma(sig, floor=1e-12):
    sig = np.asarray(sig, dtype=float)
    bad = sig < floor
    if bad.any():
        print(f'[boot] σ floor applied to {bad.sum()} cells')
        sig[bad] = floor
    return sig


In [ ]:
# 1. Import Required Libraries and Functions
import os
import sys
import urllib.request
import numpy as np
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("../scripts/entropic_spectrum.py"), '.')))
from entropic_spectrum import thet_tt

# T-HET spectrum model as a function
# (no file loading needed; use thet_tt directly)
thet_model = lambda e: thet_tt(e)

In [ ]:
# Ensure T-HET spectrum file exists
def ensure_thet_spectrum():
    import os, subprocess, sys, pathlib
    spectrum_path = pathlib.Path("data/cmb/Cl_TT_T-HET_model.txt")
    if not spectrum_path.exists():
        print("Spectrum file missing — generating via script…")
        subprocess.check_call([sys.executable, "scripts/cmb_analysis_T-HET_vs_LCDM_adjusted_II.py"])
        assert spectrum_path.exists(), "Spectrum generation failed!"
ensure_thet_spectrum()


In [ ]:
import os
import urllib.request
import numpy as np

# 1. Sigma floor to avoid division by zero or infinity in chi2
sigma = np.random.rand(100)  # This line is just for the sake of example. Remove or replace it with the actual sigma calculation.
sigma_safe = np.where(sigma < 1e-12, 1e-12, sigma)
if np.any(sigma < 1e-12):
    print("Warning: Some sigma values were floored to 1e-12 to avoid division by zero or infinity.")
# Use sigma_safe in all chi2, residual, and stats calculations below (including helper functions)
# Example: chi2 = np.sum((data - model)**2 / sigma_safe**2)
# If you have helper functions, ensure they use sigma_safe instead of sigma

# 2. Ensure Planck TT Data Availability
planck_path = "data/cmb/COM_PowerSpect_CMB-TT-full_R3.01.txt"
url = "https://pla.esac.esa.int/pla-sl/data-action?COSMOLOGY.FILE_ID=COM_PowerSpect_CMB-TT-full_R3.01.txt"
if not os.path.exists(planck_path):
    os.makedirs(os.path.dirname(planck_path), exist_ok=True)
    print(f"Downloading Planck TT data to {planck_path} ...", end=" ", flush=True)
    def reporthook(blocknum, blocksize, totalsize):
        downloaded = blocknum * blocksize
        percent = min(100, int(downloaded * 100 / totalsize)) if totalsize > 0 else 0
        print(f"{percent}%", end="\r", flush=True)
    urllib.request.urlretrieve(url, planck_path, reporthook)
    print("Download complete.")
else:
    print("Planck TT data already present.")

In [ ]:
# 3. Compute T-HET Model TT Spectrum
ell_array = np.arange(2, 2508)
Cl_TT_Thet = thet_tt(ell_array)


In [ ]:
# 4. Export T-HET Model Spectrum to File
output_path = "data/cmb/Cl_TT_T-HET_model.txt"
if not os.path.exists(output_path):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    np.savetxt(output_path, np.column_stack([ell_array, Cl_TT_Thet]))
    print(f"Saved T-HET spectrum to {output_path}")
else:
    print(f"T-HET spectrum already present at {output_path}")
